## Manhattan Grocery Store Location Suggestion

Download and import all the dependencies that will be needed.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 957.92 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00   1.49 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.24 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  28.70 MB/s
vincent-0.4.4- 100% |###################

### Downloading and Exploring dataset containing New York Boroughs and Neighbourhoods

In [3]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


Since all the relevant data is in features key, hence defining a new variable that include this data

In [4]:
neighborhoods_data = newyork_data['features']

Taking a look at the first item in this list.

In [5]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

Transforming the New York features data of nested Python dictionaries into a pandas dataframe

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#loop through the data and fill the dataframe one row at a time
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Number of Boroughs and Neighborhoods in New York

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Here we are interested in the Neighborhoods of Manhattan

In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Getting the Geographical coordinates of Manhattan

In [10]:
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Visualizing Manhattan and the neighborhoods in it.

In [11]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Utilizing Foursquare API to explore the Manhattan Neighbourhoods

In [12]:
CLIENT_ID = 'IV3RSY3ACIINHNQUJTK4GUSQLYMR54DHVTN32A5GUCCBNNNZ' # your Foursquare ID
CLIENT_SECRET = 'RO5IUTN2OKI0HPG4U0UXWIUNL43U5LWIVH3SK1PRQXECE2CK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IV3RSY3ACIINHNQUJTK4GUSQLYMR54DHVTN32A5GUCCBNNNZ
CLIENT_SECRET:RO5IUTN2OKI0HPG4U0UXWIUNL43U5LWIVH3SK1PRQXECE2CK


Creating a function to repeat the same process to all the neighborhoods in Manhattan

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Creating a new dataframe called manhattan_venues.

In [14]:
# type your answer here
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
manhattan_venues.head()

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop


Curating unique Venue Categories

In [15]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(manhattan_venues['Venue Category'].unique())

There are 326 uniques categories.
Here is the list of different categories:


['Pizza Place',
 'Yoga Studio',
 'Diner',
 'Coffee Shop',
 'Donut Shop',
 'Seafood Restaurant',
 'Department Store',
 'Tennis Stadium',
 'Gym',
 'Pharmacy',
 'Video Game Store',
 'Discount Store',
 'Supplement Shop',
 'Shoe Store',
 'Ice Cream Shop',
 'Comfort Food Restaurant',
 'Bank',
 'Steakhouse',
 'Sandwich Place',
 'Deli / Bodega',
 'Chinese Restaurant',
 'General Entertainment',
 'Cocktail Bar',
 'Greek Restaurant',
 'English Restaurant',
 'Museum',
 'Noodle House',
 'Hotel',
 'Garden Center',
 'Spa',
 'American Restaurant',
 'Bubble Tea Shop',
 'Roof Deck',
 'Tea Room',
 'Bar',
 'New American Restaurant',
 'Indie Movie Theater',
 'Salon / Barbershop',
 'Malay Restaurant',
 'Furniture / Home Store',
 'Bike Shop',
 'Vietnamese Restaurant',
 'Hotpot Restaurant',
 'Dim Sum Restaurant',
 'Italian Restaurant',
 'Jewelry Store',
 'Spanish Restaurant',
 'Music Venue',
 'Historic Site',
 'Asian Restaurant',
 'Organic Grocery',
 'Nail Salon',
 'Grocery Store',
 'Cosmetics Shop',
 'Snack 

### Analyzing each Manhattan Neighborhood using onehot encoding

In [16]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,S

Creating the restaurant category, grocery store category and needed features list to use later for integration

In [17]:
restaurant_list = ['Pizza Place',
 'Diner',
 'Coffee Shop',
 'Donut Shop',
 'Seafood Restaurant',
 'Comfort Food Restaurant',
 'Steakhouse',
 'Sandwich Place',
 'Deli / Bodega',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Greek Restaurant',
 'English Restaurant',
 'Noodle House',
 'American Restaurant',
 'Bubble Tea Shop',
 'Bar',
 'New American Restaurant',
 'Malay Restaurant',
 'Vietnamese Restaurant',
 'Hotpot Restaurant',
 'Dim Sum Restaurant',
 'Italian Restaurant',
 'Spanish Restaurant',
 'Asian Restaurant',
 'Snack Place',
 'Restaurant',
 'Dumpling Restaurant',
 'Bakery',
 'Taiwanese Restaurant',
 'Dessert Shop',
 'Shanghai Restaurant',
 'Austrian Restaurant',
 'Mexican Restaurant',
 'Ramen Restaurant',
 'Café',
 'Burger Joint',
 'Tapas Restaurant',
 'Breakfast Spot',
 'Wine Shop',
 'Caribbean Restaurant',
 'Indian Restaurant',
 'Latin American Restaurant',
 'Sushi Restaurant',
 'Wine Bar',
 'Beer Bar',
 'Arepa Restaurant',
 'Bistro',
 'Fast Food Restaurant',
 'Juice Bar',
 'Japanese Restaurant',
 'Mediterranean Restaurant',
 'Pub',
 'Gastropub',
 'BBQ Joint',
 'Japanese Curry Restaurant',
 'Beer Garden',
 'Falafel Restaurant',
 'Food & Drink Shop',
 'Cuban Restaurant',
 'Food Court',
 'French Restaurant',
 'Ethiopian Restaurant',
 'African Restaurant',
 'Bagel Shop',
 'Fried Chicken Joint',
 'Cafeteria',
 'Southern / Soul Food Restaurant',
 'Thai Restaurant',
 'Taco Place',
 'Vegetarian / Vegan Restaurant',
 'Burrito Place',
 'Hotel Bar',
 'Salad Place',
 'Turkish Restaurant',
 'German Restaurant',
 'Hot Dog Joint',
 'Peruvian Restaurant',
 'Sports Bar',
 'Afghan Restaurant',
 'Czech Restaurant',
 'Middle Eastern Restaurant',
 'Australian Restaurant',
 'South Indian Restaurant',
 'Caucasian Restaurant',
 'Szechuan Restaurant',
 'Hawaiian Restaurant',
 'Brazilian Restaurant',
 'Jewish Restaurant',
 'Scandinavian Restaurant',
 'Paella Restaurant',
 'Israeli Restaurant',
 'Udon Restaurant',
 'Eastern European Restaurant',
 'Gaming Cafe',
 'Lebanese Restaurant',
 'Moroccan Restaurant',
 'Korean Restaurant',
 'Pet Café',
 'Swiss Restaurant',
 'Soup Place',
 'Filipino Restaurant',
 'Argentinian Restaurant',
 'Whisky Bar',
 'Modern European Restaurant',
 'Karaoke Bar',
 'Wings Joint',
 'Irish Pub',
 'Belgian Restaurant',
 'Tiki Bar',
 'Kosher Restaurant',
 'Tibetan Restaurant',
 'Sake Bar',
 'Venezuelan Restaurant',
 'Molecular Gastronomy Restaurant',
 'Street Food Gathering',
 'Persian Restaurant',
 'Cambodian Restaurant',
 'Soba Restaurant',
 'Russian Restaurant',
 'Portuguese Restaurant',
 'Pakistani Restaurant']

grocery_list = ['Organic Grocery',
 'Grocery Store',
 'Market',
 'Frozen Yogurt Shop',
 'Supermarket',
 'Farmers Market',
 'Convenience Store',
 'Health Food Store',
 'Fish Market',
 'Flea Market']

needed_features = ['Neighborhood',
 'Pizza Place',
 'Diner',
 'Coffee Shop',
 'Donut Shop',
 'Seafood Restaurant',
 'Comfort Food Restaurant',
 'Steakhouse',
 'Sandwich Place',
 'Deli / Bodega',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Greek Restaurant',
 'English Restaurant',
 'Noodle House',
 'American Restaurant',
 'Bubble Tea Shop',
 'Bar',
 'New American Restaurant',
 'Malay Restaurant',
 'Vietnamese Restaurant',
 'Hotpot Restaurant',
 'Dim Sum Restaurant',
 'Italian Restaurant',
 'Spanish Restaurant',
 'Asian Restaurant',
 'Snack Place',
 'Restaurant',
 'Dumpling Restaurant',
 'Bakery',
 'Taiwanese Restaurant',
 'Dessert Shop',
 'Shanghai Restaurant',
 'Austrian Restaurant',
 'Mexican Restaurant',
 'Ramen Restaurant',
 'Café',
 'Burger Joint',
 'Tapas Restaurant',
 'Breakfast Spot',
 'Wine Shop',
 'Caribbean Restaurant',
 'Indian Restaurant',
 'Latin American Restaurant',
 'Sushi Restaurant',
 'Wine Bar',
 'Beer Bar',
 'Arepa Restaurant',
 'Bistro',
 'Fast Food Restaurant',
 'Juice Bar',
 'Japanese Restaurant',
 'Mediterranean Restaurant',
 'Pub',
 'Gastropub',
 'BBQ Joint',
 'Japanese Curry Restaurant',
 'Beer Garden',
 'Falafel Restaurant',
 'Food & Drink Shop',
 'Cuban Restaurant',
 'Food Court',
 'French Restaurant',
 'Ethiopian Restaurant',
 'African Restaurant',
 'Bagel Shop',
 'Fried Chicken Joint',
 'Cafeteria',
 'Southern / Soul Food Restaurant',
 'Thai Restaurant',
 'Taco Place',
 'Vegetarian / Vegan Restaurant',
 'Burrito Place',
 'Hotel Bar',
 'Salad Place',
 'Turkish Restaurant',
 'German Restaurant',
 'Hot Dog Joint',
 'Peruvian Restaurant',
 'Sports Bar',
 'Afghan Restaurant',
 'Czech Restaurant',
 'Middle Eastern Restaurant',
 'Australian Restaurant',
 'South Indian Restaurant',
 'Caucasian Restaurant',
 'Szechuan Restaurant',
 'Hawaiian Restaurant',
 'Brazilian Restaurant',
 'Jewish Restaurant',
 'Scandinavian Restaurant',
 'Paella Restaurant',
 'Israeli Restaurant',
 'Udon Restaurant',
 'Eastern European Restaurant',
 'Gaming Cafe',
 'Lebanese Restaurant',
 'Moroccan Restaurant',
 'Korean Restaurant',
 'Pet Café',
 'Swiss Restaurant',
 'Soup Place',
 'Filipino Restaurant',
 'Argentinian Restaurant',
 'Whisky Bar',
 'Modern European Restaurant',
 'Karaoke Bar',
 'Wings Joint',
 'Irish Pub',
 'Belgian Restaurant',
 'Tiki Bar',
 'Kosher Restaurant',
 'Tibetan Restaurant',
 'Sake Bar',
 'Venezuelan Restaurant',
 'Molecular Gastronomy Restaurant',
 'Street Food Gathering',
 'Persian Restaurant',
 'Cambodian Restaurant',
 'Soba Restaurant',
 'Russian Restaurant',
 'Portuguese Restaurant',
 'Pakistani Restaurant',
 'Organic Grocery',
 'Grocery Store',
 'Market',
 'Frozen Yogurt Shop',
 'Supermarket',
 'Farmers Market',
 'Convenience Store',
 'Health Food Store',
 'Fish Market',
 'Flea Market']

Grouping the Data by Neighborhoods and takinf sum of the individual feature values

In [18]:
manhattan_grouped = manhattan_onehot[needed_features].groupby('Neighborhood').sum().reset_index()
manhattan_grouped.head()

,Neighborhood,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market
0,Battery Park City,2,0,8,0,0,0,1,2,0,1,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,1,0,0,2,0,0,3,0,1,0,1,1,0,0,1,0,1,0,0,1,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0
1,Carnegie Hill,6,0,5,0,0,0,0,0,1,1,2,0,0,0,1,0,3,2,0,1,0,0,2,0,0,0,1,0,2,0,0,0,0,1,1,4,1,0,1,3,0,1,0,1,1,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0
2,Central Harlem,2,0,0,0,2,0,0,0,1,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,2,1,3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,Chelsea,1,0,6,0,3,0,1,1,0,0,1,0,0,1,3,0,1,1,0,0,0,0,6,0,2,0,0,0,4,0,1,0,0,1,0,2,1,2,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1
4,Chinatown,1,0,2,0,0,0,0,2,0,9,3,1,1,3,4,3,2,1,2,4,3,4,1,1,2,1,1,3,3,1,1,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0


Integrating different restaurants and different grocery store categories and taking their sum. This is to get an idea as to what is the population of restaurants vs grocery stores in each neighborhood.

In [19]:
manhattan_grouped['Total Restaurants'] = manhattan_grouped[restaurant_list].sum(axis = 1)
manhattan_grouped['Total Grocery Store'] = manhattan_grouped[grocery_list].sum(axis = 1)
manhattan_grouped

,Neighborhood,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market,Total Restaurants,Total Grocery Store
0,Battery Park City,2,0,8,0,0,0,1,2,0,1,0,0,0,0,2,0,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,1,0,0,2,0,0,3,0,1,0,1,1,0,0,1,0,1,0,0,1,0,2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,39,3
1,Carnegie Hill,6,0,5,0,0,0,0,0,1,1,2,0,0,0,1,0,3,2,0,1,0,0,2,0,0,0,1,0,2,0,0,0,0,1,1,4,1,0,1,3,0,1,0,1,1,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,59,3
2,Central Harlem,2,0,0,0,2,0,0,0,1,2,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,2,1,3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,27,1
3,Chelsea,1,0,6,0,3,0,1,1,0,0,1,0,0,1,3,0,1,1,0,0,0,0,6,0,2,0,0,0,4,0,1,0,0,1,0,2,1,2,1,1,0,1,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,1,51,4
4,Chinatown,1,0,2,0,0,0,0,2,0,9,3,1,1,3,4,3,2,1,2,4,3,4,1,1,2,1,1,3,3,1,1,1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,64,2
5,Civic Center,0,1,3,0,0,0,0,3,0,0,3,0,0,0,2,1,1,1,0,0,0,0,3,0,1,0,0,0,5,0,0,0,0,1,1,1,0,0,0,2,0,1,0,3,1,0,0,0,0,0,1,0,0,0,0,0,0,2,0,1,0,3,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,48,1
6,Clinton,1,1,4,0,1,0,1,2,1,1,1,0,0,1,4,0,1,2,

### Running k-means to Cluster Manhattan Neighborhoods into 5 Clusters

In [20]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(manhattan_grouped_clustering)
kmeans.labels_

array([2, 4, 2, 0, 4, 0, 0, 2, 3, 4, 0, 4, 3, 0, 2, 2, 3, 2, 4, 2, 0, 2, 1,
       0, 4, 2, 3, 4, 1, 2, 1, 0, 0, 0, 3, 0, 3, 0, 4, 3], dtype=int32)

Showing centers of each Cluster for each venue category

In [21]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = manhattan_grouped_clustering.columns
means_df.index = ['C0','C1','C2','C3','C4']
means_df

,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market,Total Restaurants,Total Grocery Store
C0,1.916667,0.583333,3.166667,3.333333e-01,0.833333,3.469447e-18,0.833333,1.500000,1.000000,1.000000,1.333333,0.750000,3.469447e-18,1.666667e-01,2.666667,0.166667,1.166667,0.750000,8.333333e-02,2.500000e-01,1.387779e-17,0.083333,3.416667e+00,0.333333,0.416667,0.083333,0.250000,0.000000,2.666667,3.469447e-18,0.666667,8.333333e-02,6.938894e-18,1.750000,2.500000e-01,2.250000,0.916667,0.500000,1.666667e-01,1.416667,0.416667,1.250000,5.833333e-01,1.416667,0.833333,2.500000e-01,8.333333e-02,6.938894e-18,0.166667,0.916667,0.916667,0.666667,0.333333,0.250000,-2.775558e-17,1.387779e-17,1.666667e-01,2.500000e-01,1.387779e-17,0.250000,0.083333,1.50,0.166667,1.387779e-17,0.333333,2.500000e-01,3.469447e-18,1.387779e-17,0.583333,0.166667,7.500000e-01,8.333333e-02,0.25,3.333333e-01,8.333333e-02,8.333333e-02,1.387779e-17,0.166667,0.083333,3.469447e-18,3.469447e-18,8.333333e-02,8.333333e-02,3.469447e-18,8.333333e-02,2.500000e-01,2.500000e-01,8.333333e-02,8.333333e-02,3.469447e-18,8.333333e-02,1.666667e-01,3.469447e-18,3.469447e-18,3.469447e-18,8.333333e-02,6.938894e-18,2.500000e-01,6.938894e-18,1.387779e-17,3.469447e-18,1.387779e-17,8.333333e-02,8.333333e-02,8.333333e-02,2.775558e-17,1.666667e-01,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,6.938894e-18,3.469447e-18,8.333333e-02,3.469447e-18,8.333333e-02,8.333333e-02,3.469447e-18,8.333333e-02,8.333333e-02,3.469447e-18,6.938894e-18,8.333333e-01,2.500000e-01,0.333333,0.250000,0.166667,0.083333,0.166667,8.333333e-02,8.333333e-02,49.750000,2.250000
C1,0.333333,0.333333,1.333333,3.333333e-01,0.333333,3.333333e-01,0.333333,1.000000,0.666667,0.000000,0.333333,0.333333,-3.469447e-18,0.000000e+00,0.333333,0.000000,1.333333,0.000000,-1.387779e-17,5.551115e-17,0.000000e+00,0.000000,4.440892e

Taking the sum of all the features to create 'Total Sum', after excluding the 'Total Grocery store' column. This would help to sort the clusters based on the demand for a new grocery store considering the population of already existing grocery stores.

In [22]:
col_list = list(manhattan_grouped_clustering)
col_list.remove('Total Grocery Store')
means_df['Total Sum'] = means_df[col_list].sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Pizza Place,Diner,Coffee Shop,Donut Shop,Seafood Restaurant,Comfort Food Restaurant,Steakhouse,Sandwich Place,Deli / Bodega,Chinese Restaurant,Cocktail Bar,Greek Restaurant,English Restaurant,Noodle House,American Restaurant,Bubble Tea Shop,Bar,New American Restaurant,Malay Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Dim Sum Restaurant,Italian Restaurant,Spanish Restaurant,Asian Restaurant,Snack Place,Restaurant,Dumpling Restaurant,Bakery,Taiwanese Restaurant,Dessert Shop,Shanghai Restaurant,Austrian Restaurant,Mexican Restaurant,Ramen Restaurant,Café,Burger Joint,Tapas Restaurant,Breakfast Spot,Wine Shop,Caribbean Restaurant,Indian Restaurant,Latin American Restaurant,Sushi Restaurant,Wine Bar,Beer Bar,Arepa Restaurant,Bistro,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Mediterranean Restaurant,Pub,Gastropub,BBQ Joint,Japanese Curry Restaurant,Beer Garden,Falafel Restaurant,Food & Drink Shop,Cuban Restaurant,Food Court,French Restaurant,Ethiopian Restaurant,African Restaurant,Bagel Shop,Fried Chicken Joint,Cafeteria,Southern / Soul Food Restaurant,Thai Restaurant,Taco Place,Vegetarian / Vegan Restaurant,Burrito Place,Hotel Bar,Salad Place,Turkish Restaurant,German Restaurant,Hot Dog Joint,Peruvian Restaurant,Sports Bar,Afghan Restaurant,Czech Restaurant,Middle Eastern Restaurant,Australian Restaurant,South Indian Restaurant,Caucasian Restaurant,Szechuan Restaurant,Hawaiian Restaurant,Brazilian Restaurant,Jewish Restaurant,Scandinavian Restaurant,Paella Restaurant,Israeli Restaurant,Udon Restaurant,Eastern European Restaurant,Gaming Cafe,Lebanese Restaurant,Moroccan Restaurant,Korean Restaurant,Pet Café,Swiss Restaurant,Soup Place,Filipino Restaurant,Argentinian Restaurant,Whisky Bar,Modern European Restaurant,Karaoke Bar,Wings Joint,Irish Pub,Belgian Restaurant,Tiki Bar,Kosher Restaurant,Tibetan Restaurant,Sake Bar,Venezuelan Restaurant,Molecular Gastronomy Restaurant,Street Food Gathering,Persian Restaurant,Cambodian Restaurant,Soba Restaurant,Russian Restaurant,Portuguese Restaurant,Pakistani Restaurant,Organic Grocery,Grocery Store,Market,Frozen Yogurt Shop,Supermarket,Farmers Market,Convenience Store,Health Food Store,Fish Market,Flea Market,Total Restaurants,Total Grocery Store,Total Sum
C3,2.428571,1.000000,3.714286,-2.775558e-17,1.428571,3.469447e-18,1.142857,1.142857,1.142857,1.142857,1.428571,0.428571,3.469447e-18,4.285714e-01,1.571429,0.142857,3.428571,0.714286,1.387779e-17,8.571429e-01,0.000000e+00,0.000000,5.714286e+00,0.571429,0.428571,0.142857,0.857143,0.142857,1.857143,3.469447e-18,1.000000,1.428571e-01,6.938894e-18,1.714286,7.142857e-01,1.714286,1.714286,0.285714,2.857143e-01,1.142857,0.428571,1.571429,2.857143e-01,3.142857,2.428571,1.428571e-01,2.857143e-01,6.938894e-18,0.000000,0.714286,2.000000,0.857143,1.000000,0.142857,2.857143e-01,1.428571e-01,1.387779e-17,2.775558e-17,1.428571e-01,0.571429,0.000000,2.00,0.000000,0.000000e+00,1.285714,1.428571e-01,3.469447e-18,1.428571e-01,1.285714,0.428571,1.571429e+00,6.938894e-18,0.00,4.285714e-01,7.142857e-01,2.857143e-01,2.857143e-01,0.428571,0.285714,1.428571e-01,1.428571e-01,5.714286e-01,1.428571e-01,1.428571e-01,6.938894e-18,0.000000e+00,1.428571e-01,6.938894e-18,2.857143e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,2.857143e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,1.428571e-01,6.938894e-18,3.469447e-18,2.857143e-01,0.000000e+00,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,1.428571e-01,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,3.469447e-18,1.428571e-01,3.469447e-18,3.469447e-18,3.469447e-18,1.428571e-01,4.285714e-01,2.775558e-17,0.000000,0.000000,0.428571,0.000000,0.285714,3.469447e-18,6.938894e-18,68.285714,1.285714,137.857143
C4,2.875000,0.375000,4.000000,-2.775558e-17,0.875000,3.469447e-18,1.000000,1.375000,0.500000,2.125000,2.625000,0.375000,1.250000e-01,5.000000e-01,2.375000,0.875000,1.375000,1.375000,3.750000e-01,1.125000e+00,3.750000e-0

### Suggestion for the top 3 Manhattan Neighborhood Clusters to open a new Grocery store

Best Cluster is C3

Second Best Cluster is C4

Third Best Cluster is C0

#### Inserting "kmeans.labels_" into the Original Manhattan DataFrame 

In [23]:
manhattan_grouped1 = manhattan_grouped.set_index('Neighborhood')
manhattan_cluster = pd.DataFrame([manhattan_grouped1.index, kmeans.labels_]).T
manhattan_cluster.columns = ['Neighborhood', 'Cluster Labels']
manhattan_cluster.head()

,Neighborhood,Cluster Labels
0,Battery Park City,2
1,Carnegie Hill,4
2,Central Harlem,2
3,Chelsea,0
4,Chinatown,4


#### Neighbourhoods in the Top 3 Clusters

Cluster 3

In [24]:
manhattan_cluster[manhattan_cluster['Cluster Labels'] == 3]

,Neighborhood,Cluster Labels
8,East Village,3
12,Greenwich Village,3
16,Lenox Hill,3
26,Murray Hill,3
34,Turtle Bay,3
36,Upper West Side,3
39,Yorkville,3


Cluster 4

In [25]:
manhattan_cluster[manhattan_cluster['Cluster Labels'] == 4]

,Neighborhood,Cluster Labels
1,Carnegie Hill,4
4,Chinatown,4
9,Financial District,4
11,Gramercy,4
18,Little Italy,4
24,Midtown South,4
27,Noho,4
38,West Village,4


Cluster 0

In [26]:
manhattan_cluster[manhattan_cluster['Cluster Labels'] == 0]

,Neighborhood,Cluster Labels
3,Chelsea,0
5,Civic Center,0
6,Clinton,0
10,Flatiron,0
13,Hamilton Heights,0
20,Manhattan Valley,0
23,Midtown,0
31,Sutton Place,0
32,Tribeca,0
33,Tudor City,0


### Clusters on the Manhattan map. 

#### The markdowns colored in Red, Purple and Blue are the Top 3 neighborhood clusters where a new grocery store is suggested to be opened.

In [27]:
manhattan_merged = manhattan_data
manhattan_merged['Cluster Labels'] = kmeans.labels_
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,Manhattan,Marble Hill,40.876551,-73.910660,2
1,Manhattan,Chinatown,40.715618,-73.994279,4
2,Manhattan,Washington Heights,40.851903,-73.936900,2
3,Manhattan,Inwood,40.867684,-73.921210,0
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4


In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

kclusters = 5

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Thank You! This brings us to the end of the 'Manhattan Grocery Store Location Suggestion' Notebook...